<a href="https://colab.research.google.com/github/Hanbin-git/practice/blob/main/upbit2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pyupbit
import time
import datetime
import os
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import numpy as np

# API 키 설정
access = "Nw2bQfQdCj5yIVKBlMlxIqbv0zWXCSZk2YoUxsCL"
secret = "4K6ZgQWUQ8AXqphJO7x5n0sGUjr9WLWVUxqqVQTy"
upbit = pyupbit.Upbit(access, secret)


SAVE_DIR = "E:/crypto_log"
os.makedirs(SAVE_DIR, exist_ok=True)
log_path = os.path.join(SAVE_DIR, "trade_log.txt")
chart_path = os.path.join(SAVE_DIR, "asset_chart.png")

cash = 1_000_000
holding = 0
buy_price = 0
prev_price = None
last_trade_time = None

buy_fee = 0.0005
sell_fee = 0.0005

rsi_buy_threshold = 35
rsi_sell_threshold = 65

time_log, asset_log = [], []

def get_indicators(ticker):
    df = pyupbit.get_ohlcv(ticker, interval="minute60", count=200)
    close = df['close']

    # RSI
    delta = close.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(14).mean()
    avg_loss = loss.rolling(14).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    # MA20
    ma20 = close.rolling(window=20).mean()

    # MACD
    ema12 = close.ewm(span=12, adjust=False).mean()
    ema26 = close.ewm(span=26, adjust=False).mean()
    macd = ema12 - ema26
    signal = macd.ewm(span=9, adjust=False).mean()

    return rsi.iloc[-1], ma20.iloc[-1], macd.iloc[-1], signal.iloc[-1], close.iloc[-1]

ticker = "KRW-XRP"

while True:
    try:
        now = datetime.datetime.now()
        rsi, ma20, macd, signal, price = get_indicators(ticker)
        now_str = now.strftime('%Y-%m-%d %H:%M:%S')

        time_ok = (last_trade_time is None) or ((now - last_trade_time).seconds > 60 * 30)

        msg = f"[{now_str}] [관망] RSI: {rsi:.2f}, 가격: {price:.1f}, 총 자산: {cash + holding * price:,.0f}원"

        # 매수 조건: RSI, MA, MACD 모두 만족해야 함
        if rsi < rsi_buy_threshold and price > ma20 and macd > signal and cash > 0 and time_ok:
            amount = cash * 0.3 * (1 - buy_fee)  # 30% 분할매수
            qty = amount / price
            holding += qty
            buy_price = price if buy_price == 0 else (buy_price + price) / 2
            cash -= amount / (1 - buy_fee)
            last_trade_time = now
            msg = f"[{now_str}] ✅ [매수] RSI: {rsi:.2f}, 가격: {price}, 수량: {qty:.4f}"

        # 매도 조건
        elif rsi > rsi_sell_threshold and macd < signal and holding > 0 and time_ok:
            qty = holding
            amount = qty * price * (1 - sell_fee)
            profit = amount - (buy_price * qty)
            cash = amount
            holding = 0
            last_trade_time = now
            msg = f"[{now_str}] ✅ [매도] RSI: {rsi:.2f}, 가격: {price}, 수익: {profit:,.0f}원"

        print(msg)
        with open(log_path, "a", encoding="utf-8") as f:
            f.write(msg + "\n")

        # 자산 그래프
        total_asset = cash + holding * price
        time_log.append(now_str)
        asset_log.append(total_asset)

        if len(asset_log) > 1:
            plt.figure(figsize=(10, 4))
            plt.plot(time_log, asset_log, marker='o')
            plt.title("자산 변화 추이")
            plt.xlabel("시간")
            plt.ylabel("KRW")
            plt.xticks(rotation=45)
            plt.grid(True)
            plt.tight_layout()
            plt.savefig(chart_path)
            plt.close()

    except Exception as e:
        print(f"[에러] {e}")

    # 🔁 1초 주기로 체크 (주의: 실운영 시엔 너무 빠름)
    time.sleep(10)